# Process Analysis Homework
For this assignment you will be reviewing a memory image infected with APT malware. The image has been provided for you to download and is named `APT.img` For this assignment we will just be focused on reviwing processes and determining if any of them need more detailed analysis. First you must identify what profile to use with your analysis with the `imageinfo` command

## Profile Identification

In [1]:
python C:/Users/roman/volatility/vol.py -f C:/Users/roman/Downloads/APT/APT.img imageinfo

Volatility Foundation Volatility Framework 2.6
INFO    : volatility.debug    : Determining profile based on KDBG search...
          Suggested Profile(s) : WinXPSP2x86, WinXPSP3x86 (Instantiated with WinXPSP2x86)
                     AS Layer1 : IA32PagedMemoryPae (Kernel AS)
                     AS Layer2 : FileAddressSpace (C:\Users\roman\Downloads\APT\APT.img)
                      PAE type : PAE
                           DTB : 0x319000L
                          KDBG : 0x80545b60L
          Number of Processors : 1
     Image Type (Service Pack) : 3
                KPCR for CPU 0 : 0xffdff000L
             KUSER_SHARED_DATA : 0xffdf0000L
           Image date and time : 2009-05-05 19:28:57 UTC+0000
     Image local date and time : 2009-05-05 15:28:57 -0400


Now that you know the profile use that to review the following process analysis with volatility:
 * `plist`
 * `pstree`
 * `psscan`
 * `psxview`
 
 Document any of your findings of supicious processes in line and make sure to document each analysis execution.
 
 ## PSLIST

In [2]:
python C:/Users/roman/volatility/vol.py -f C:/Users/roman/Downloads/APT/APT.img --profile=WinXPSP2x86 pslist

Volatility Foundation Volatility Framework 2.6
Offset(V)  Name                    PID   PPID   Thds     Hnds   Sess  Wow64 Start                          Exit                          
---------- -------------------- ------ ------ ------ -------- ------ ------ ------------------------------ ------------------------------
0x823c8830 System                    4      0     55      254 ------      0                                                              
0x8230aad8 smss.exe                564      4      3       19 ------      0 2009-04-16 16:10:01 UTC+0000                                 
0x822ca2c0 csrss.exe               636    564     10      356      0      0 2009-04-16 16:10:06 UTC+0000                                 
0x81f63020 winlogon.exe            660    564     16      502      0      0 2009-04-16 16:10:06 UTC+0000                                 
0x81f22020 services.exe            704    660     15      254      0      0 2009-04-16 16:10:06 UTC+0000                     

### PID 796 iexplore.exe and PID 840 cmd.exe looks suspicious because of dates.

## PSTREE

In [3]:
python C:/Users/roman/volatility/vol.py -f C:/Users/roman/Downloads/APT/APT.img --profile=WinXPSP2x86 pstree

Volatility Foundation Volatility Framework 2.6
Name                                                  Pid   PPid   Thds   Hnds Time
-------------------------------------------------- ------ ------ ------ ------ ----
 0x823c8830:System                                      4      0     55    254 1970-01-01 00:00:00 UTC+0000
. 0x8230aad8:smss.exe                                 564      4      3     19 2009-04-16 16:10:01 UTC+0000
.. 0x81f63020:winlogon.exe                            660    564     16    502 2009-04-16 16:10:06 UTC+0000
... 0x81f22020:services.exe                           704    660     15    254 2009-04-16 16:10:06 UTC+0000
.... 0x81f739b0:svchost.exe                          1088    704     70   1445 2009-04-16 16:10:07 UTC+0000
..... 0x81f96220:wscntfy.exe                         1260   1088      1     39 2009-04-16 16:10:22 UTC+0000
.... 0x81da4590:svchost.exe                           968    704     10    241 2009-04-16 16:10:07 UTC+0000
.... 0x81dc2570:VMwareService

### PID 796 and 840. Same as before.

## PSSCAN

In [4]:
python C:/Users/roman/volatility/vol.py -f C:/Users/roman/Downloads/APT/APT.img --profile=WinXPSP2x86 psscan

Volatility Foundation Volatility Framework 2.6
Offset(P)          Name                PID   PPID PDB        Time created                   Time exited                   
------------------ ---------------- ------ ------ ---------- ------------------------------ ------------------------------
0x0000000001f33628 alg.exe             464    704 0x08c002e0 2009-04-16 16:10:21 UTC+0000                                 
0x0000000001fa4590 svchost.exe         968    704 0x08c00100 2009-04-16 16:10:07 UTC+0000                                 
0x0000000001fa71a8 explorer.exe       1672   1624 0x08c00200 2009-04-16 16:10:10 UTC+0000                                 
0x0000000001fbdda0 iexplore.exe        796    884 0x08c00320 2009-05-05 19:28:28 UTC+0000                                 
0x0000000001fc1a78 VMwareUser.exe     2004   1672 0x08c00160 2009-04-16 16:10:11 UTC+0000                                 
0x0000000001fc2570 VMwareService.e    1032    704 0x08c00280 2009-04-16 16:10:16 UTC+0000   

## PSXVIEW

In [5]:
python C:/Users/roman/volatility/vol.py -f C:/Users/roman/Downloads/APT/APT.img --profile=WinXPSP2x86 psxview

Volatility Foundation Volatility Framework 2.6
Offset(P)  Name                    PID pslist psscan thrdproc pspcid csrss session deskthrd ExitTime
---------- -------------------- ------ ------ ------ -------- ------ ----- ------- -------- --------
0x02163020 winlogon.exe            660 True   True   True     True   True  True    True     
0x02122020 services.exe            704 True   True   True     True   True  True    True     
0x0211a650 ctfmon.exe             2020 True   True   True     True   True  True    True     
0x01fa71a8 explorer.exe           1672 True   True   True     True   True  True    True     
0x0252c020 svchost.exe            1140 True   True   True     True   True  True    True     
0x0204d648 cmd.exe                 840 True   True   True     True   True  True    True     
0x01fc1a78 VMwareUser.exe         2004 True   True   True     True   True  True    True     
0x02054da0 svchost.exe             884 True   True   True     True   True  True    True     
0x02196

### anything that isn't "true" all across should be looked into.

## Summary of Findings
Please summarize all your findings here with cleaned up documented evidence

In [6]:
python C:/Users/roman/volatility/vol.py -f C:/Users/roman/Downloads/APT/APT.img --profile=WinXPSP2x86 connections

Volatility Foundation Volatility Framework 2.6
Offset(V)  Local Address             Remote Address            Pid
---------- ------------------------- ------------------------- ---
0x81e611f8 192.168.157.10:1053       218.85.133.23:89          796


### PID shows up again. Port 89 not normal and IP address is from Asia Pacific Network Information Centre. 

In [7]:
python C:/Users/roman/volatility/vol.py -f C:/Users/roman/Downloads/APT/APT.img --profile=WinXPSP2x86 connscan

Volatility Foundation Volatility Framework 2.6
Offset(P)  Local Address             Remote Address            Pid
---------- ------------------------- ------------------------- ---
0x0205ece0 192.168.157.10:1050       222.128.1.2:443           1672
0x020611f8 192.168.157.10:1053       218.85.133.23:89          796
0x032c01f8 192.168.157.10:1053       218.85.133.23:89          796
0x0337dce0 192.168.157.10:1050       222.128.1.2:443           1672
0x08a4ace0 192.168.157.10:1050       222.128.1.2:443           1672
0x18200ce0 192.168.157.10:1050       222.128.1.2:443           1672


### 1672 PID explorer.exe should not be connecting directly to the internet. IP address is again from Asia.

In [8]:
python C:/Users/roman/volatility/vol.py -f C:/Users/roman/Downloads/APT/APT.img --profile=WinXPSP2x86 sockets

Volatility Foundation Volatility Framework 2.6
Offset(V)       PID   Port  Proto Protocol        Address         Create Time
---------- -------- ------ ------ --------------- --------------- -----------
0x822ef510      796   1053      6 TCP             0.0.0.0         2009-05-05 19:29:06 UTC+0000
0x81fa9008     1212   1900     17 UDP             192.168.157.10  2009-04-16 16:10:20 UTC+0000
0x822d25b0        4      0     47 GRE             0.0.0.0         2009-05-05 19:28:35 UTC+0000
0x81e59748      716    500     17 UDP             0.0.0.0         2009-04-16 16:10:16 UTC+0000
0x822d7220        4    139      6 TCP             192.168.157.10  2009-04-16 16:10:00 UTC+0000
0x8206ce98      464   1032      6 TCP             127.0.0.1       2009-04-16 16:10:21 UTC+0000
0x82305008        4    445      6 TCP             0.0.0.0         2009-04-16 16:10:00 UTC+0000
0x81df5298      968    135      6 TCP             0.0.0.0         2009-04-16 16:10:07 UTC+0000
0x82305258        4    137     17 UDP

### PID 796 shows up again and is now listening on port 1053. PID 769's Create Time is a couple weeks after all of the other PID's except PID 4 which is System. 31 seconds after PID 4 comes on PID comes on too.

In [9]:
python C:/Users/roman/volatility/vol.py -f C:/Users/roman/Downloads/APT/APT.img --profile=WinXPSP2x86 sockscan

Volatility Foundation Volatility Framework 2.6
Offset(P)       PID   Port  Proto Protocol        Address         Create Time
---------- -------- ------ ------ --------------- --------------- -----------
0x01f354c8     1088    123     17 UDP             127.0.0.1       2009-04-16 16:10:20 UTC+0000
0x01ff5298      968    135      6 TCP             0.0.0.0         2009-04-16 16:10:07 UTC+0000
0x02059748      716    500     17 UDP             0.0.0.0         2009-04-16 16:10:16 UTC+0000
0x021a9008     1212   1900     17 UDP             192.168.157.10  2009-04-16 16:10:20 UTC+0000
0x021a9658     1088    123     17 UDP             192.168.157.10  2009-04-16 16:10:20 UTC+0000
0x0226ce98      464   1032      6 TCP             127.0.0.1       2009-04-16 16:10:21 UTC+0000
0x023514f8     1212   1900     17 UDP             127.0.0.1       2009-04-16 16:10:20 UTC+0000
0x023554c8     1088   1028     17 UDP             127.0.0.1       2009-04-16 16:10:20 UTC+0000
0x02365320      716      0    255 Res

### PID 796 shows up twice here with different volatility offsets